In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
from sklearn.utils import resample
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn import model_selection
from sklearn import metrics
from sklearn.decomposition import PCA
from scipy.stats import zscore
import matplotlib.pyplot as plt 

In [4]:
colnames = ['preg', 'glu', 'bp', 'sft', 'ins', 'bmi', 'dpf', 'age', 'outcome']
prima_df = pd.read_csv("pima-indians-diabetes.data",names=colnames)

In [0]:
X=prima_df[['preg', 'glu', 'bp', 'sft', 'ins', 'bmi', 'dpf', 'age']]
Y=prima_df['outcome']

In [0]:
type(X)

pandas.core.frame.DataFrame

In [0]:
sc=StandardScaler()
X=sc.fit_transform(X)

In [0]:
from sklearn.model_selection import GridSearchCV
knn=KNeighborsClassifier()
param={'n_neighbors':np.arange(1,50),'weights':['uniform','distance']}
GS=GridSearchCV(knn,param,cv=5,scoring='recall')
GS.fit(X,Y)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='warn', n_jobs=None,
             param_grid={'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]),
                         'weights': ['uniform', 'distance']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='recall', verbose=0)

In [0]:
from sklearn.model_selection import KFold
kf=KFold(n_splits=5,shuffle=True,random_state=0)
param2={'n_estimators':np.arange(1,25)}
GS2=GridSearchCV(NB_bag,param2,cv=kf,scoring='roc_auc')
GS2.fit(X,Y)

GridSearchCV(cv=KFold(n_splits=5, random_state=0, shuffle=True),
             error_score='raise-deprecating',
             estimator=BaggingClassifier(base_estimator=GaussianNB(priors=None,
                                                                   var_smoothing=1e-09),
                                         bootstrap=True,
                                         bootstrap_features=False,
                                         max_features=1.0, max_samples=1.0,
                                         n_estimators=10, n_jobs=None,
                                         oob_score=False, random_state=0,
                                         verbose=0, warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'n_estimators': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='roc_auc', verb

In [0]:
GS2.best_params_

{'n_estimators': 24}

In [0]:
LR=LogisticRegression()
NB=GaussianNB()
#update and redifine your knn model
KNN=KNeighborsClassifier(n_neighbors=12, weights='distance')

In [0]:
from sklearn.ensemble import BaggingClassifier
LR_bag=BaggingClassifier(base_estimator=LR,n_estimators=25,random_state=0)
NB_bag=BaggingClassifier(base_estimator=NB,n_estimators=24,random_state=0)
KNN_bag=BaggingClassifier(base_estimator=KNN,n_estimators=80,random_state=0)
DT_reg=DecisionTreeClassifier(max_depth=4,criterion='entropy',random_state=0)
DT_bag=BaggingClassifier(n_estimators=10,random_state=0)
RF=RandomForestClassifier(n_estimators=130,criterion='entropy',random_state=0)
#Boosting models
LR_boost=AdaBoostClassifier(base_estimator=LR,n_estimators=170)
NB_boost=AdaBoostClassifier(base_estimator=NB,n_estimators=350)
RF_boost=AdaBoostClassifier(base_estimator=RF,n_estimators=100)
DT_boost=AdaBoostClassifier(n_estimators=100)
GB_boost=GradientBoostingClassifier(n_estimators=100)
#Stacked Model
stacked = VotingClassifier(estimators = [('Bagged_LR',LR_bag),('BoostedRF', RF_boost), ('GBoost', GB_boost)],voting='soft')

In [0]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [0]:
from sklearn.model_selection import KFold
kf=KFold(n_splits=5,shuffle=True,random_state=2)
for model, name in zip([boost_LR,base_rf,boost_dt,gb_model,stacked], ['BoostLR','RF','BoostedDT','GradientBoost','stacked']):
    k=0
    recall=np.zeros((2,5))
    prec=np.zeros((2,5))
    fscore=np.zeros((2,5))
    for train,test in kf.split(X,Y):
        Xtrain,Xtest=X[train,:],X[test,:]
        Ytrain,Ytest=Y[train],Y[test]
        model.fit(Xtrain,Ytrain)
        Y_predict=model.predict(Xtest)
        cm=metrics.confusion_matrix(Ytest,Y_predict)
        for i in np.arange(0,2):
            recall[i,k]=cm[i,i]/cm[i,:].sum()
        for i in np.arange(0,2):
            prec[i,k]=cm[i,i]/cm[:,i].sum()
        k=k+1
    for row in np.arange(0,2):
        for col in np.arange(0,5):
            fscore[row,col]=2*(recall[row,col]*prec[row,col])/(recall[row,col]+prec[row,col])
    print("f1_weighted for Healthy: %0.02f (+/- %0.5f) [%s]" % (np.mean(fscore[0,:]), np.var(fscore[0,:],ddof=1), name ))   
    print("f1_weighted for Diabetic: %0.02f (+/- %0.5f) [%s]" % (np.mean(fscore[1,:]), np.var(fscore[1,:],ddof=1), name ))   
    

f1_weighted for Healthy: 0.83 (+/- 0.00048) [BoostLR]
f1_weighted for Diabetic: 0.62 (+/- 0.00222) [BoostLR]
f1_weighted for Healthy: 0.81 (+/- 0.00029) [RF]
f1_weighted for Diabetic: 0.62 (+/- 0.00171) [RF]
f1_weighted for Healthy: 0.81 (+/- 0.00028) [BoostedDT]
f1_weighted for Diabetic: 0.60 (+/- 0.00683) [BoostedDT]
f1_weighted for Healthy: 0.82 (+/- 0.00037) [GradientBoost]
f1_weighted for Diabetic: 0.62 (+/- 0.00248) [GradientBoost]
f1_weighted for Healthy: 0.81 (+/- 0.00020) [stacked]
f1_weighted for Diabetic: 0.62 (+/- 0.00166) [stacked]


In [27]:
from sklearn.model_selection import KFold
from sklearn.metrics import roc_curve, auc
kf=KFold(n_splits=5,shuffle=True,random_state=0)
for model, name in zip([LR,LR_bag,NB,NB_bag,KNN,KNN_bag,DT_reg,DT_bag,RF,RF_boost,stacked], 
      ['LR','BaggedLR','NB','BaggedNB','KNN','BaggedKNN','DT_Reg','BaggedDT','RF','RF_boost','Stacked']):
    roc_auc=[]
    for train,test in kf.split(X,Y):
        Xtrain,Xtest=X.iloc[train,:],X.iloc[test,:]
        Ytrain,Ytest=Y.iloc[train],Y.iloc[test]
        model.fit(Xtrain,Ytrain)
        Y_predict=model.predict(Xtest)
        #cm=metrics.confusion_matrix(Ytest,Y_predict)
        fpr,tpr, _ = roc_curve(Ytest,Y_predict)
        roc_auc.append(auc(fpr, tpr))
    print("AUC scores: %0.02f (+/- %0.5f) [%s]" % (np.mean(roc_auc),
                                    np.var(roc_auc,ddof=1), name ))   
    

AUC scores: 0.71 (+/- 0.00126) [LR]
AUC scores: 0.71 (+/- 0.00074) [BaggedLR]
AUC scores: 0.71 (+/- 0.00237) [NB]
AUC scores: 0.71 (+/- 0.00248) [BaggedNB]
AUC scores: 0.70 (+/- 0.00085) [KNN]
AUC scores: 0.69 (+/- 0.00099) [BaggedKNN]
AUC scores: 0.72 (+/- 0.00219) [DT_Reg]
AUC scores: 0.70 (+/- 0.00341) [BaggedDT]
AUC scores: 0.73 (+/- 0.00148) [RF]
AUC scores: 0.73 (+/- 0.00233) [RF_boost]
AUC scores: 0.72 (+/- 0.00233) [Stacked]


In [0]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [35]:
from sklearn.model_selection import KFold
from sklearn.metrics import roc_curve, auc
kf=KFold(n_splits=5,shuffle=True,random_state=0)
for model, name in zip([LR,LR_boost,NB,NB_boost,DT_boost,RF,RF_boost,GB_boost,stacked], 
      ['LR','BoostedLR','NB','BoostedNB','BoostedDT','RF','BoostedRF','GradientBoost','stacked']):
    roc_auc=[]
    for train,test in kf.split(X,Y):
        Xtrain,Xtest=X.iloc[train,:],X.iloc[test,:]
        Ytrain,Ytest=Y.iloc[train],Y.iloc[test]
        model.fit(Xtrain,Ytrain)
        Y_predict=model.predict(Xtest)
        #cm=metrics.confusion_matrix(Ytest,Y_predict)
        fpr,tpr, _ = roc_curve(Ytest,Y_predict)
        roc_auc.append(auc(fpr, tpr))
    print("AUC scores: %0.02f (+/- %0.5f) [%s]" % (np.mean(roc_auc),
                                    np.var(roc_auc,ddof=1), name ))   
    

AUC scores: 0.71 (+/- 0.00126) [LR]
AUC scores: 0.71 (+/- 0.00163) [BoostedLR]
AUC scores: 0.71 (+/- 0.00237) [NB]
AUC scores: 0.57 (+/- 0.00200) [BoostedNB]
AUC scores: 0.70 (+/- 0.00363) [BoostedDT]
AUC scores: 0.73 (+/- 0.00148) [RF]
AUC scores: 0.72 (+/- 0.00129) [BoostedRF]
AUC scores: 0.73 (+/- 0.00300) [GradientBoost]
AUC scores: 0.73 (+/- 0.00173) [stacked]


In [0]:

(0.72-0.71)/0.72


0.013888888888888902

In [0]:
(0.00088-0.00073)/0.00088

0.17045454545454553